In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
import cv2
import numpy as np
from PIL import Image
from tensorflow.compat.v1 import InteractiveSession
import tensorflow
config = tensorflow.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tensorflow.compat.v1.Session(config=config)

c:\users\loren\anaconda3\envs\tensorflowgpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5



In [2]:
model_pred = load_model("model_face.h5")
model_pred.load_weights("model_face_weights.h5")

In [5]:
import cv2

# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# To capture video from webcam. 
cap = cv2.VideoCapture(0)
framerate = cap.get(2)
# To use a video file as input 
# cap = cv2.VideoCapture('filename.mp4')

while True:
    # Read the frame
    _, img = cap.read()
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect the faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    
    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        crop_img = img[y:y+h, x:x+w]
        
        image_from_array = Image.fromarray(crop_img, 'RGB')
        size_image = image_from_array.resize((224, 224))
        
        image = preprocess_input(np.array(size_image))
        image = np.expand_dims(image, axis=0)
        
        prediction = model_pred.predict(image).argmax(axis = -1)
        
        if prediction == 0:
            label = "With mask"
        else:
            label = "No mask"

        color = (0, 255, 0) if label == "With mask" else (0, 0, 255)

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(img, label, (x, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2,)
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        
       
    cv2.imshow('img', img)
    key = cv2.waitKey(1) & 0xFF
    # Display
    # Stop if escape key is pressed

    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
        
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 